In [1]:
!pip install flask-ngrok

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.17.1


In [3]:
!pip install nest_asyncio pyngrok Flask werkzeug
!pip install flask_cors

In [4]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token('2jpFZIDm2XWMOjfZl6QGKciSAbY_3jo9X2RDhN9TaBgk97G94')

In [5]:
import nest_asyncio
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from joblib import load
import keras
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.serving import run_simple
from pyngrok import ngrok
from threading import Thread

In [6]:
@keras.saving.register_keras_serializable(package="MyLayers")
class L2Normalization(Layer):
    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=1)

custom_objects = {
    "L2Normalization": L2Normalization
}

In [7]:
model = tf.keras.models.load_model('my_model.keras',custom_objects=custom_objects)

In [8]:
user_scaler=load('UserScaler.joblib')
movie_scaler=load('MovieScaler.joblib')
rating_scaler=load('MinMaxRatingScaler.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
def load_dataset(scalerItem,scalerUser):
    # Load the CSV file
    df = pd.read_csv("updated_api_data.csv")
    dfUser=pd.read_csv("user_file_forApi.csv")
    item_train = scalerItem.transform(df.iloc[:,3:20])
    user_train= scalerUser.transform(dfUser)
    info=df.iloc[:,[1,2,4,20]]
    return [info,user_train[:,3:],item_train[:,2:]]

In [10]:
nest_asyncio.apply()
app = Flask(__name__)

CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Parse the input JSON request
        input_data = request.json.get("data")

        # Ensure input data is a valid numpy array
        if input_data is None or not isinstance(input_data, list):
            return jsonify({"error": "Invalid input. Expected a list of numbers."}), 400

        # Convert input data to a NumPy array
        input_array = np.array(input_data).reshape(1, -1)  # Adjust reshape based on model input requirements

        columns = ['3.95', '4.25', '0.0', '0.0.1', '4.0.1',
           '4.12', '4.0.2', '4.04', '0.0.2', '3.0', '4.0.3', '0.0.3', '3.88', '3.89']
        df = pd.DataFrame(input_array, columns=columns)
        tempDf=pd.read_csv('temp_user_data.csv')
        df=pd.concat([tempDf,df],axis=1)

        # Replicate rows until the length is 847
        replicated_df = pd.concat([df] * (847 // len(df)), ignore_index=True)  # Full replication
        remaining_rows = 847 - len(replicated_df)  # Remaining rows to reach 847
        if remaining_rows > 0:
            replicated_df = pd.concat([replicated_df, df.iloc[:remaining_rows]], ignore_index=True)

        user_train=user_scaler.transform(replicated_df)


        info,_,item_train=load_dataset(movie_scaler,user_scaler)

        # Make predictions using the model
        predictions = model.predict([user_train[:,3:],item_train]) #[:,3:]
        predictions= rating_scaler.inverse_transform(predictions)

        # Convert predictions to a list
        predictions_list = predictions.flatten().tolist()  # Flatten if predictions are multi-dimensional

        # Format predictions with additional dataframe information
        formatted_predictions = []
        for i, pred in enumerate(predictions_list):
            if i >= len(info):  # Ensure we don't access out-of-range indices
                break

            # Get the corresponding row from the dataframe
            row = info.iloc[i]
            label = ""
            if pred >= 4.5:
                label = "Highly Recommended"
            elif 3.5 <= pred < 4.5:  # Combine conditions correctly
                label = "Good Movie for You"
            elif 2.5 <= pred < 3.5:
                label = "Average"
            else:
                label = "Not for You"
            # Build the JSON object for the prediction
            prediction_object = {
                "prediction": pred,
                "label":label,
                "title": row["title"],
                "genres": row["genres"],
                "year": int(row["2003"]),  # Adjust the column name if needed
                "image_url": row["Image URL"]
            }
            formatted_predictions.append(prediction_object)

        sorted_data = sorted(formatted_predictions, key=lambda x: x["prediction"], reverse=True)

        # Return sorted predictions as a JSON object
        return jsonify(sorted_data)

        # Return formatted predictions as a JSON object
        #return jsonify(formatted_predictions)

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Function to run Flask in a separate thread
def run_flask():
    run_simple('0.0.0.0', 8999, app, use_reloader=False)

# Start Flask server in a separate thread
flask_thread = Thread(target=run_flask)
flask_thread.start()

# Start ngrok tunnel
public_url = ngrok.connect(8999)  # Port should match the Flask server port
print('Public URL:', public_url)

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8999
 * Running on http://172.28.0.2:8999
INFO:werkzeug:Press CTRL+C to quit


Public URL: NgrokTunnel: "https://3927-34-23-69-56.ngrok-free.app" -> "http://localhost:8999"
